In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
import sys

sys.path.append(os.path.abspath("common"))
from evaluate import evaluate_predictions
from preprocess import getdfs, get_char_columns, one_hot_encode, find_common_characters, remove_common_characters, process_strings
from evaluate import get_classification_report, evaluate_predictions
from models import predict_logistic_regression
from models import predict_svc

In [5]:
emoticon_train_df, emoticon_test_df = getdfs(data='emoticon')
text_seq_train_df, text_seq_test_df = getdfs(data='text_seq')
feature_train_df, feature_test_df = getdfs(data='feature')


In [10]:
import pandas as pd
import numpy as np
from helper import print_accuracy, remove_common_characters, get_char_columns, find_common_characters, process_strings
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import CountVectorizer


### Emoticon

In [11]:
train_df = pd.read_csv('datasets/train/train_emoticon.csv')
valid_df = pd.read_csv('datasets/valid/valid_emoticon.csv')
test_df = pd.read_csv('datasets/test/test_emoticon.csv')

train_df['input_emoticon'] = remove_common_characters(train_df['input_emoticon'])
valid_df['input_emoticon'] = remove_common_characters(valid_df['input_emoticon'])
test_df['input_emoticon'] = remove_common_characters(test_df['input_emoticon'])

train_df = get_char_columns(train_df)
valid_df = get_char_columns(valid_df)
test_df = get_char_columns(test_df)

y_train = train_df['label']
y_valid = valid_df['label']

train_df = train_df.drop('label', axis=1)
valid_df = valid_df.drop('label', axis = 1)

oh_encoder = OneHotEncoder(handle_unknown = 'ignore')
oh_encoder.fit(train_df)


x_train = pd.DataFrame(oh_encoder.transform(train_df).toarray())
x_valid = pd.DataFrame(oh_encoder.transform(valid_df).toarray())
x_test = pd.DataFrame(oh_encoder.transform(test_df).toarray())

params = {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}

model = LogisticRegression(**params, max_iter= 10000)
model.fit(x_train, y_train)
y_valid_pred = model.predict(x_valid)


In [7]:
# emoticon_train_df['input_emoticon'] = strings_after_removal
# emoticon_test_df['input_emoticon'] = remove_common_characters(emoticon_test_df['input_emoticon'].values)

In [20]:
# train_df = get_char_columns(emoticon_train_df)
# test_df = get_char_columns(emoticon_test_df)
# emoti_x_train, emoti_x_valid, emoti_y_train, emoti_y_valid = one_hot_encode(train_df, test_df)
# # print(x_train.shape, x_valid.shape, y_train.shape, y_valid.shape)

(7080, 501) (489, 501)


In [22]:
params = {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}

model = sklearn.linear_model.LogisticRegression(**params, max_iter= 10000)

In [23]:
model.fit(emoti_x_train, emoti_y_train)

LogisticRegression(C=10, max_iter=10000, penalty='l1', solver='liblinear')

In [12]:
emoti_train_pred_proba = model.predict_proba(x_train)
emoti_test_pred_proba = model.predict_proba(x_valid)

In [20]:
# import numpy as np
# from sklearn.neural_network import MLPClassifier
# from sklearn.metrics import classification_report

# # Train the MLP model (assuming you have already trained it)
# mlp_model = MLPClassifier(hidden_layer_sizes=(16,), max_iter=1000)
# mlp_model.fit(x_train, y_train)

# # Predict probabilities for each class (softmax output for all samples)
# y_proba_mlp = mlp_model.predict_proba(x_valid)

# # Display softmax probabilities for all samples
# print("Softmax probabilities for all samples:\n", y_proba_mlp)

Softmax probabilities for all samples:
 [[9.56090405e-04 9.99043910e-01]
 [2.13558318e-08 9.99999979e-01]
 [1.00000000e+00 2.04260712e-27]
 [9.99999997e-01 2.90756128e-09]
 [4.55286105e-06 9.99995447e-01]
 [2.53751464e-10 1.00000000e+00]
 [1.01035301e-08 9.99999990e-01]
 [9.99999999e-01 1.13935484e-09]
 [9.99999997e-01 2.75185118e-09]
 [6.54488819e-09 9.99999993e-01]
 [1.00000000e+00 3.43064038e-17]
 [1.15298500e-06 9.99998847e-01]
 [9.82407809e-01 1.75921908e-02]
 [1.30887186e-07 9.99999869e-01]
 [2.07771105e-01 7.92228895e-01]
 [1.00000000e+00 1.88464854e-17]
 [2.44340861e-06 9.99997557e-01]
 [3.30598429e-08 9.99999967e-01]
 [6.53761090e-09 9.99999993e-01]
 [1.00000000e+00 4.77980850e-22]
 [9.99999996e-01 3.95652556e-09]
 [3.52970075e-09 9.99999996e-01]
 [1.00000000e+00 2.32213370e-19]
 [1.00000000e+00 4.02264460e-34]
 [9.88309467e-01 1.16905334e-02]
 [1.00000000e+00 1.24799399e-19]
 [1.54078185e-03 9.98459218e-01]
 [2.06117807e-06 9.99997939e-01]
 [9.99999911e-01 8.91409257e-08]
 [2

### Feature

In [13]:
train_df = np.load('datasets/train/train_feature.npz', allow_pickle=True)
valid_df = np.load('datasets/valid/valid_feature.npz', allow_pickle=True)
test_df = np.load('datasets/test/test_feature.npz', allow_pickle=True)

x_train = train_df['features']
y_train = train_df['label']
x_valid = valid_df['features']
y_valid = valid_df['label']
x_test = test_df['features']

x_train = x_train.reshape(x_train.shape[0], -1)
x_valid = x_valid.reshape(x_valid.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)

params = {'C': 10.0, 'fit_intercept': True, 'penalty': 'l2', 'solver': 'lbfgs'}
model = LogisticRegression(**params, max_iter= 1000)
model.fit(x_train, y_train)
y_valid_pred = model.predict(x_valid)

# print_accuracy(y_valid, y_valid_pred)

# generate_submission_txt(model, x_test, file_name='pred_deepfeat.txt')


In [28]:
import sklearn.linear_model


model = sklearn.linear_model.LogisticRegression(max_iter=10000, random_state=42)

In [29]:
model.fit(train_X, train_Y)

LogisticRegression(max_iter=10000, random_state=42)

In [14]:
feature_train_pred_proba = model.predict_proba(x_train)
feature_test_pred_proba = model.predict_proba(x_valid)

In [41]:
# feat_y_pred = predict_proba_logistic_regression(train_X, train_Y, valid_X)

In [ ]:
# feat_y_pred

### Text

In [23]:
import os
import sys

import pandas as pd
import numpy as np
import typing

sys.path.append(os.path.abspath("../common"))  # add path to common functions
from evaluate import evaluate_predictions
from preprocess import getdfs, get_char_columns
from preprocess import one_hot_encode
from models import predict_logistic_regression
from models import predict_xgboost

In [32]:

repeat_emos = ['🙼', '🛐', '🙯', '😛', '😣', '😑', '🚼']

repeat_emo_code = {
     '🙼' : '284',
     '🛐' : '464', 
     '🙯' : '262',
     '😛' : '15436', 
     '😣' : '614',
     '😑' : '1596', 
     '🚼' : '422'
}

In [15]:
emo_train_df = pd.read_csv('datasets/train/train_emoticon.csv')
repeat_emos = find_common_characters(emo_train_df['input_emoticon'])

train_df = pd.read_csv('datasets/train/train_text_seq.csv')
valid_df = pd.read_csv('datasets/valid/valid_text_seq.csv')
test_df = pd.read_csv('datasets/test/test_text_seq.csv')

train_df["input_str"] = process_strings(train_df["input_str"])
valid_df["input_str"] = process_strings(valid_df["input_str"])
test_df["input_str"] = process_strings(test_df["input_str"])

y_train = train_df['label'].values
y_valid = valid_df['label'].values

_x_train = train_df['input_str'].values
_x_valid = valid_df['input_str'].values
_x_test = test_df['input_str'].values

vectorizer = CountVectorizer(ngram_range=(3, 5), analyzer='char')  # Extract n-grams

x_train = vectorizer.fit_transform(_x_train)
x_valid = vectorizer.transform(_x_valid)
x_test = vectorizer.transform(_x_test)

params  = {'colsample_bytree': 1.0, 'eval_metric': 'logloss', 'gamma': 0.2, 'learning_rate': 0.1, 'max_depth': 7, 'min_child_weight': 3, 'n_estimators': 500, 'subsample': 1.0}
model = XGBClassifier(**params)
model.fit(x_train, y_train)
y_valid_pred = model.predict(x_valid)


In [34]:

text_seq_train_df['input_str'] = process_strings(text_seq_train_df['input_str'])
text_seq_test_df['input_str'] = process_strings(text_seq_test_df['input_str'])

In [35]:
num_feat = 15

def get_columns (df) :
    for i in range(num_feat):
        df[f'c_{i}'] = df['input_str'].apply(lambda x : x[i])
    return df.drop(columns = ['input_str'])

text_seq_train_df = get_columns(text_seq_train_df)
text_seq_test_df = get_columns(text_seq_test_df)

In [36]:
text_seq_test_df.head()

,label,c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,c_12,c_13,c_14
0,1,3,3,4,1,4,1,3,9,0,1,0,0,0,0,0
1,1,1,2,5,0,6,2,3,2,7,3,1,0,4,0,0
2,0,4,5,4,3,6,3,3,6,5,2,7,1,3,0,0
3,0,1,1,2,0,4,3,7,1,3,1,3,2,0,0,0
4,1,3,3,2,4,2,3,6,8,4,5,7,0,5,0,0


In [37]:
# Feature engineering (example: one-hot encoding)

train_df, valid_df, y_train, y_valid = one_hot_encode(text_seq_train_df, text_seq_test_df)

(7080, 142) (489, 142)


In [38]:
x_train = train_df.values
x_valid = valid_df.values

In [39]:
from xgboost import XGBClassifier

model = XGBClassifier()

In [40]:
model.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [16]:
text_seq_train_pred_proba = model.predict_proba(x_train)
text_seq_test_pred_proba = model.predict_proba(x_valid)

In [17]:
hard_pred = np.array((np.argmax(emoti_test_pred_proba, axis = 1) + np.argmax(feature_test_pred_proba, axis = 1) + np.argmax(text_seq_test_pred_proba, axis = 1)) >= 2, dtype = np.float64)
rep = get_classification_report(hard_pred, y_valid)
print(rep)
print(sklearn.metrics.accuracy_score(hard_pred, y_valid))

              precision    recall  f1-score   support

     Class 0       0.98      0.99      0.98       250
     Class 1       0.99      0.98      0.98       239

    accuracy                           0.98       489
   macro avg       0.98      0.98      0.98       489
weighted avg       0.98      0.98      0.98       489

0.983640081799591


In [19]:
soft_pred = np.argmax(emoti_test_pred_proba + feature_test_pred_proba + text_seq_test_pred_proba, axis = 1)
rep = get_classification_report(soft_pred, y_valid)
print(rep)
print(sklearn.metrics.accuracy_score(soft_pred, y_valid))

              precision    recall  f1-score   support

     Class 0       0.98      0.98      0.98       250
     Class 1       0.98      0.97      0.98       239

    accuracy                           0.98       489
   macro avg       0.98      0.98      0.98       489
weighted avg       0.98      0.98      0.98       489

0.9795501022494888


In [94]:
import torch
import torch.nn as nn
import torch.optim as optim

train_inputs = torch.tensor(np.hstack([emoti_train_pred_proba[:, 1].reshape(-1, 1), feature_train_pred_proba[:, 1].reshape(-1, 1), text_seq_train_pred_proba[:, 1].reshape(-1,1)]), dtype = torch.float32)
train_targets = torch.tensor(y_train.values.reshape(-1, 1), dtype = torch.float32)
test_inputs = torch.tensor(np.hstack([emoti_test_pred_proba[:, 1].reshape(-1, 1), feature_test_pred_proba[:, 1].reshape(-1, 1), text_seq_test_pred_proba[:, 1].reshape(-1,1)]), dtype = torch.float32)
test_targets = torch.tensor(y_valid.values.reshape(-1, 1), dtype = torch.float32)

In [97]:
# Define a simple neural network with a single linear layer
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.linear = nn.Linear(3, 1)  # 3 inputs, 1 output

    def forward(self, x):
        return torch.sigmoid(self.linear(x))  # Apply sigmoid for binary classification


In [139]:
import sklearn

model = sklearn.linear_model.LogisticRegression(max_iter=10000, random_state=42)

model.fit(train_inputs.numpy(), train_targets.numpy().flatten())

train_accuracy = sklearn.metrics.accuracy_score(train_targets, model.predict(train_inputs))
test_accuracy = sklearn.metrics.accuracy_score(test_targets, model.predict(test_inputs))
print(f"{train_accuracy=:.4f}, {test_accuracy=:.4f}")

train_accuracy=1.0000, test_accuracy=0.9673


In [142]:
model.coef_

array([[7.25131344, 4.83503164, 4.15379776]])

In [102]:

# Create an instance of the neural network
model = SimpleNN()

# Define loss function (binary cross entropy) and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Example of generating some dummy data
# 10 samples, 3 features per sample
# Binary targets (0 or 1)

# Example of a forward pass (without training)
outputs = model(train_inputs)

# Loss calculation (no backward pass or optimization here)
loss = criterion(outputs, train_targets)

# Print the outputs and loss
print("Outputs:", outputs)
print("Loss:", loss.item())

Outputs: tensor([[0.6103],
        [0.5886],
        [0.5526],
        ...,
        [0.6010],
        [0.5759],
        [0.6023]], grad_fn=<SigmoidBackward0>)
Loss: 0.6806437969207764


In [130]:
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass
    model.train()
    train_outputs = model(train_inputs)
    loss = criterion(train_outputs, train_targets)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss every 20 epochs
    
    model.eval()
    train_outputs = model(train_inputs)
    val_outputs = model(test_inputs)
    train_accuracy = torch.mean(((train_outputs > 0.5) == train_targets).float())
    val_accuracy = torch.mean(((val_outputs > 0.5) == test_targets).float())
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.2f}, Train Accuracy: {train_accuracy.item():.4f}, Validation Accuracy: {val_accuracy.item():.4f}')    

Epoch 1/100, Loss: 0.12, Train Accuracy: 0.9976, Validation Accuracy: 0.9652
Epoch 2/100, Loss: 0.12, Train Accuracy: 0.9976, Validation Accuracy: 0.9652
Epoch 3/100, Loss: 0.12, Train Accuracy: 0.9976, Validation Accuracy: 0.9652
Epoch 4/100, Loss: 0.12, Train Accuracy: 0.9976, Validation Accuracy: 0.9652
Epoch 5/100, Loss: 0.12, Train Accuracy: 0.9976, Validation Accuracy: 0.9652
Epoch 6/100, Loss: 0.12, Train Accuracy: 0.9976, Validation Accuracy: 0.9652
Epoch 7/100, Loss: 0.12, Train Accuracy: 0.9976, Validation Accuracy: 0.9652
Epoch 8/100, Loss: 0.12, Train Accuracy: 0.9976, Validation Accuracy: 0.9652
Epoch 9/100, Loss: 0.12, Train Accuracy: 0.9976, Validation Accuracy: 0.9652
Epoch 10/100, Loss: 0.12, Train Accuracy: 0.9976, Validation Accuracy: 0.9652
Epoch 11/100, Loss: 0.12, Train Accuracy: 0.9976, Validation Accuracy: 0.9652
Epoch 12/100, Loss: 0.12, Train Accuracy: 0.9976, Validation Accuracy: 0.9652
Epoch 13/100, Loss: 0.12, Train Accuracy: 0.9976, Validation Accuracy: 0.

In [67]:
model.state_dict()

OrderedDict([('linear.weight', tensor([[2.0077, 1.9070]])),
             ('linear.bias', tensor([-1.9103]))])

In [92]:
idk = (np.array(feature_test_pred_proba[:, 1] > 0.5, dtype = np.float32) == y_valid.values)[np.invert(np.array(emoti_test_pred_proba[:, 1] > 0.5, dtype = np.float32) == y_valid.values)]
idk.sum(), idk.shape[0]

(9, 14)

In [93]:
idk_val = (np.array(feature_train_pred_proba[:, 1] > 0.5, dtype = np.float32) == y_train.values)[np.invert(np.array(emoti_train_pred_proba[:, 1] > 0.5, dtype = np.float32) == y_train.values)]
idk_val.sum(), idk_val.shape[0]

(3, 4)

In [ ]:
[np.invert(np.array(emoti_test_pred_proba[:, 1] > 0.5, dtype = np.float32) == y_valid.values)]

In [71]:
y_valid.values

array([1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0,

In [143]:
from xgboost import XGBClassifier

model = XGBClassifier()

model.fit(train_inputs.numpy(), train_targets.numpy().flatten())

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [144]:
train_accuracy = sklearn.metrics.accuracy_score(train_targets, model.predict(train_inputs))
test_accuracy = sklearn.metrics.accuracy_score(test_targets, model.predict(test_inputs))
print(f"{train_accuracy=:.4f}, {test_accuracy=:.4f}")

train_accuracy=1.0000, test_accuracy=0.9673
